This code was originally written by Karen Perez, for GBT C-Band data. I generalized this code to calculate the spike channels for other bands at GBT. It now takes in an h5/filterbank file and its corresponding .dat file

In [12]:
f = bl.Waterfall(h5_files[9])
f.info()

blimpy.io.base_reader WARNING  Selection size of 19.25 GB, exceeding our size limit 1.00 GB. Instance created, header loaded, but data not loaded, please try another (t,v) selection.

--- File Info ---
DIMENSION_LABELS : [b'frequency' b'feed_id' b'time']
        az_start :                              0.0
       data_type :                                1
            fch1 :                1926.26953125 MHz
            foff :      -2.7939677238464355e-06 MHz
      machine_id :                               20
           nbits :                               32
          nchans :                        322961408
            nifs :                                1
     source_name :                         HIP82860
         src_dej :                     65:08:06.008
         src_raj :                      16:56:01.98
    telescope_id :                                6
           tsamp :                     18.253611008
   tstart (ISOT) :          2016-10-03T22:09:21.000
    tstart (MJD) 

In [17]:
1926.26953125 + 322961409*-2.7939677238464355e-06

1023.9257784560323

In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import blimpy as bl
import tqdm
import glob

In [2]:
dat_files = [
    "dat_files/GBT_57523_69379_HIP17147_fine.dat", 
    "dat_files/GBT_57606_50058_HIP20901_fine.dat",
    "dat_files/GBT_57456_02669_HIP39826_fine.dat",
    "dat_files/GBT_57803_80733_HIP4436_fine.dat",  
    "dat_files/GBT_57599_55512_HIP45493_fine.dat", 
    "dat_files/GBT_57459_34297_HIP65352_fine.dat", 
    "dat_files/GBT_57650_54573_HIP66704_fine.dat", 
    "dat_files/GBT_57523_22406_HIP74981_fine.dat", 
    "dat_files/GBT_57680_15520_HIP7981_fine.dat",  
    "dat_files/GBT_57664_79761_HIP82860_fine.dat" 
]

In [3]:
h5_files = [
    "/mnt_blpd7/datax/dl/GBT_57523_69379_HIP17147_fine.h5",
    "/mnt_blpd7/datax/dl/GBT_57606_50058_HIP20901_fine.h5",
    "/mnt_blpd7/datax/dl/GBT_57456_02669_HIP39826_fine.h5",
    "/mnt_blpd7/datax2/dl/GBT_57803_80733_HIP4436_fine.h5",
    "/mnt_blpd7/datax/dl/GBT_57599_55512_HIP45493_fine.h5",
    "/mnt_blpd7/datax/dl/GBT_57459_34297_HIP65352_fine.h5",
    "/mnt_blpd7/datax2/dl/GBT_57650_54573_HIP66704_fine.h5",
    "/mnt_blpd7/datax/dl/GBT_57523_22406_HIP74981_fine.h5",
    "/mnt_blpd7/datax2/dl/GBT_57680_15520_HIP7981_fine.h5",
    "/mnt_blpd7/datax2/dl/GBT_57664_79761_HIP82860_fine.h5"
]

In [14]:
def grab_parameters(h5_file):
    """takes h5 file of GBT data and returns a list of channels and nfpc

    returns channels_list, fch1, foff, nfpc"""
    #read h5 file and grab frequency info from the header"
    test_file = h5_file
    fb = bl.Waterfall(test_file, load_data=False)
    head = fb.file_header
    
    fch1 = head["fch1"]
    foff = head["foff"]
    nchans=head["nchans"]
    fch1=float(fch1-(foff/2.0))
    
    nfpc=(1500.0/512.0)/abs(foff)
    channels_list=range(0,nchans+1)
    
    return channels_list, fch1, foff, nfpc

The `spike_channels` function is not a very efficient cell. It would be worth looking into to see if there are more efficient ways to make the list. 

It seems like we are just making a list that increments at some intervals (hopefully a periodic interval), so it would be worth seeing if we can use a numpy function to make it faster.

In [13]:
spike_channels_list[0], spike_channels_list[-1]

(524288.0, 338649581879296.0)

In [5]:
def spike_channels(channels_list, nfpc):
    """makes a spike channels list given a list of channels"""
    spike_channels_list=[]
    for i in channels_list: #should be smaller ~512 -> variable somewhere to store this that can be changed course channels list? 
        spike_channel=(nfpc/2.0)+(nfpc*i)
        spike_channels_list.append(spike_channel)
    print ('spike channels list done')
    return spike_channels_list

In [6]:
def freqs_fine_channels(spike_channels_list, fch1, foff):
    """docstring here"""
    freqs_fine_channels_list=[]
    for index, value in enumerate(spike_channels_list):
        freq_fine_channel=fch1+foff*value
        if freq_fine_channel>0:
            #print ('freq_fine_channel', freq_fine_channel)
            #num = str(freq_fine_channel)
            #i = num.index(".")
            #freq_fine_channel = num[:i + 7]
            freq_fine_channel=round(freq_fine_channel, 6)
            freqs_fine_channels_list.append(freq_fine_channel)
        else:
            break
    #print ('freqs_fine_channels list done', freqs_fine_channels_list)
    print ('end')
    np.save('freqs_fine_channels_list2.npy', freqs_fine_channels_list)
    return freqs_fine_channels_list

In [7]:
def wrapper_function(dat_file, freqs_fine_channels_list):
    """a wrapper function to encapsulate a function that I want to call multiple times"""
    file = dat_file

    if True:#"gpuspec.0000.dat" in file and ("A00_0025" in file or "C01_0026" in file or "C07_0027" in file or "A00_0028" in file or "C01_0029" in file or "C07_0030" in file or "A00_0031" in file or "C01_0032" in file):
    #if file.endswith("gpuspec.0000.dat") and "A00_0025" in file:    
        datfile_curr=file
        print ('datfile_curr', datfile_curr)
        #data=np.loadtxt(datfile_curr)
        file_contents = []
        # open file you wish to read
        with open(datfile_curr, 'r') as infile:
            for line in infile:
                file_contents.append(line)
        with open(datfile_curr+'new.dat', 'w') as outfile:
            for index, row in enumerate(file_contents):
                #row=row.split('\t')
                #print ('row', row)
                if index==0 or index==1 or index==2 or index==3 or index==4 or index==5 or index==6 or index==7 or index==8:
                    #newrow=row.strip('\n')
                    newrow=row.strip('\t')
                    #print ('row', newrow)
                    outfile.write(newrow)       
                else:
                    newrow=row.split('\t')
                    #print ('row', row)
                    row=row.split('\t')
                    #print ('row_postsplit', row)
                    freq=float(newrow[4])-(foff/2.0)
                    startfreq=float(newrow[6])-(foff/2.0)
                    endfreq=float(newrow[7])-(foff/2.0)
                    #freq=freq.strip(' ')
                    freq=round(freq, 6)
                    startfreq=round(startfreq,6)
                    endfreq=round(endfreq,6)
                    #print ('freq', freq)
                    #row[3]=str(freq)
                    #row[4]=str(freq)
                    #print ('rowwithadjfreq', row)
                    #string='\t'
                    #row = [i + string for i in row]

                    minfreq=(float(freq)-0.000001)
                    maxfreq=(float(freq)+0.000001)
                    #print ('minfreq', minfreq)
                    #print ('maxfreq', maxfreq)
                    #i= minfreq.index(".")
                    #minfreq = minfreq[:i + 7]
                    minfreq=round(minfreq,6)
                    #j= maxfreq.index(".")
                    #maxfreq = maxfreq[:j + 7]
                    maxfreq=round(maxfreq,6)
                    #print ('minfreq', minfreq)
                    #print ('maxfreq', maxfreq)
                    freq=str(freq)
                    minfreq=str(minfreq)
                    maxfreq=str(maxfreq)
                    if len((freq).split('.')[1])<6:
                        freq=format(float(freq), '.6f')
                    row[3]=str(freq)
                    row[4]=str(freq)
                    row[6]=str(startfreq)
                    row[7]=str(endfreq)
                    string='\t'
                    for index,value in enumerate(row[:-1]):
                        newvalue=value+string
                        row[index]=newvalue
                    #row = [i + string for i in row[:-1]]
                    if len((minfreq).split('.')[1])<6:
                        minfreq=format(float(minfreq), '.6f')
                    if len((maxfreq).split('.')[1])<6:
                        maxfreq=format(float(maxfreq), '.6f')
                    #print ('freq', freq)
                    #print ('minfreq', minfreq)
                    #print ('maxfreq', maxfreq)
                    if float(freq) in freqs_fine_channels_list:
                        pass#print ('bad freq', freq)
                    elif float(minfreq) in freqs_fine_channels_list:
                        pass#print ('bad min freq', minfreq)
                    elif float(maxfreq) in freqs_fine_channels_list:
                        pass#print ('bad max freq', maxfreq)
                    else:
                        glue='  '
                        #row=row.format()
                        row=glue.join(row)
                        #row=row.format()
                        #print ('finalrow', row)
                        outfile.write(str(row))
    print("done!\n")

In [15]:
chan_list, fch1, foff, nfpc = grab_parameters(h5_files[0])

blimpy.io.base_reader INFO     Skipping loading data ...


In [9]:
spike_channels_list = spike_channels(chan_list, nfpc)

spike channels list: 100%|██████████| 318230529/318230529 [04:30<00:00, 1175579.81it/s]

spike channels list done


In [10]:
len(chan_list)

322961409

In [10]:
freqs_fine_channels_list = freqs_fine_channels(spike_channels_list,fch1, foff)

end


In [11]:
wrapper_function(dat_files[0], freqs_fine_channels_list)

datfile_curr dat_files/GBT_57523_69379_HIP17147_fine.dat
done!


## Now to generalize this one step further

I will make a for loop that passes throught the whole `dat_files` and `h5_files` lists and performs these functions on each one

In [8]:
for i in range(len(dat_files)):
    chan_list, fch1, foff, nfpc = grab_parameters(h5_files[i])
    spike_channels_list = spike_channels(chan_list, nfpc)
    freqs_fine_channels_list = freqs_fine_channels(spike_channels_list,fch1, foff)
    wrapper_function(dat_files[i], freqs_fine_channels_list)

blimpy.io.base_reader WARNING  Selection size of 18.97 GB, exceeding our size limit 1.00 GB. Instance created, header loaded, but data not loaded, please try another (t,v) selection.
spike channels list done
end
datfile_curr dat_files/GBT_57523_69379_HIP17147_fine.dat
done!
blimpy.io.base_reader WARNING  Selection size of 19.25 GB, exceeding our size limit 1.00 GB. Instance created, header loaded, but data not loaded, please try another (t,v) selection.
spike channels list done
end
datfile_curr dat_files/GBT_57606_50058_HIP20901_fine.dat
done!
blimpy.io.base_reader WARNING  Selection size of 15.77 GB, exceeding our size limit 1.00 GB. Instance created, header loaded, but data not loaded, please try another (t,v) selection.
spike channels list done
end
datfile_curr dat_files/GBT_57456_02669_HIP39826_fine.dat
done!
blimpy.io.base_reader WARNING  Selection size of 19.25 GB, exceeding our size limit 1.00 GB. Instance created, header loaded, but data not loaded, please try another (t,v) sel